In [1]:
import pandas as pd
import json
import numpy as np
with open("all_results.jsonl") as f:
    for line in f:
        result = json.loads(line)
        query_id = result["query_id"]
        pilot_time = result["pilot_time"]
        multi_time = result["multi_table_sample_time"]
        single_time = result["single_table_sample_time"]
        exact_time = result["exact_time"]
        multi_result = pd.DataFrame(result["multi_table_sample_result"])
        single_result = pd.DataFrame(result["single_table_sample_result"])
        exact_result = pd.DataFrame(result["exact_result"])
        # find the column is not of string type
        print(query_id)
        if multi_result.empty:
            print("multi_result is empty")
        else:
            non_string_columns = []
            for column in multi_result.columns:
                if single_result[column].dtype != "object":
                    non_string_columns.append(column)
            error = np.abs(single_result[non_string_columns] - exact_result[non_string_columns]) / exact_result[non_string_columns]
            max_error = error.max().max()
            print("multi table sample error: ", max_error*100)
        if single_result.empty:
            print("single_result is empty")
        else:
            non_string_columns = []
            for column in single_result.columns:
                if single_result[column].dtype != "object":
                    non_string_columns.append(column)
            error = np.abs(single_result[non_string_columns] - exact_result[non_string_columns]) / exact_result[non_string_columns]
            max_error = error.max().max()
            print("single table sample error: ",max_error*100)


7
multi_result is empty
single table sample error:  2.5550871780898645
7
multi_result is empty
single table sample error:  3.2624262760909994
7
multi_result is empty
single table sample error:  1.5449144363776939
7
multi_result is empty
single table sample error:  4.080527660241057
7
multi_result is empty
single table sample error:  2.4036281982803
7
multi_result is empty
single table sample error:  1.8266726523669425
7
multi_result is empty
single table sample error:  1.9479121924692877
7
multi_result is empty
single table sample error:  2.901449887488992
7
multi_result is empty
single table sample error:  0.8540107144340395
7
multi_result is empty
single table sample error:  1.5785637140355826
8
multi_result is empty
single_result is empty
8
multi_result is empty
single_result is empty
8
multi_result is empty
single_result is empty
8
multi_result is empty
single_result is empty
8
multi_result is empty
single_result is empty
8
multi_result is empty
single_result is empty
8
multi_resul